In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py

ModuleNotFoundError: No module named 'chart_studio'

In [3]:
import chart_studio
chart_studio.tools.set_credentials_file(username='kmohan_ts', api_key='DQQyWqXWsC1Cma8ld4Ud')

ModuleNotFoundError: No module named 'chart_studio'

In [4]:
energy_data = pd.read_csv('~/Documents/Projects/covid-energy-burden/energy_data_with_predictions_new.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Documents/Projects/covid-energy-burden/energy_data_with_predictions_new.csv'

In [ ]:
energy_data.columns

In [ ]:
energy_data.head()

In [ ]:
pre_covid_error = energy_data.groupby(['state','sector']).apply(get_pre_covid_errors).reset_index().rename(columns={0:'pre_covid_error'})
post_covid_error = energy_data.groupby(['state','sector']).apply(get_post_covid_errors).reset_index().rename(columns={0:'post_covid_error'})

In [ ]:
pre_post_error = pre_covid_error.merge(post_covid_error,on=['state','sector'])

In [ ]:
energy_data = energy_data.merge(pre_post_error,on=['state','sector'])

In [ ]:
energy_data['pre_post_diff'] = energy_data['post_covid_error'] - energy_data['pre_covid_error']

In [ ]:
energy_data['anno_text'] = ['<b>+'+str(np.round(x,2))+'%</b>' if x>0 else '<b>'+str(np.round(x,2))+'%</b>' for x in energy_data['pre_post_diff']]
energy_data['anno_color'] = ['green' if x>0 else 'red' for x in energy_data['pre_post_diff']]

In [ ]:
list(energy_data[(energy_data.state=='AZ')&(energy_data.sector=="RES")].anno_color)[0]

In [ ]:
states = energy_data[~(energy_data.state =='USA')].state.unique()

In [ ]:
states = ['USA']

In [ ]:
states.extend(energy_data[~(energy_data.state =='USA')].state.unique())

In [ ]:
energy_data['state_name'] = [abbrev_us_state[s] for s in energy_data['state']]

In [ ]:
# fig = px.line(energy_data[(energy_data.state=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)], x="date", y="pct_error_20", title='Error in prediction')
# fig.add_shape(type='line',x0='2020-02-15',x1='2020-02-15')

# # fig = go.Figure(go.Scatter(x=energy_data[(energy_data.state=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)].date,
# #                            y=np.round(energy_data[(energy_data.state=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)].pct_error_20,2),mode='lines+markers',marker_color='#27a3aa'))
fig = go.Figure()
fig.add_trace(go.Scatter(x=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)].date,
                         y=np.round(energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)].pct_error_20,2),
                         mode='lines+markers',marker_color='#27a3aa',visible=True))
# fig = px.scatter(energy_data[(energy_data.state=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)],
#                 x='date',
#                 y='pct_error_20'
#                 )
fig.add_shape(type='line',
              x0='2020-02-15',x1='2020-02-15',
              y0=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)].pct_error_20.min()*1.2,
              y1=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")&(energy_data.year >= 2019)].pct_error_20.max()*1.2,
              line=dict(color='#f76d23', width=3, dash="dot"))
fig.add_shape(type='line',
              x0='2019-01-01',x1='2020-02-15',
              y0=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")].pre_covid_error.min(),
              y1=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")].pre_covid_error.min(),
              line=dict(color='red', width=2, dash="dash"))
fig.add_shape(type='line',
              x0='2020-02-15',x1='2020-10-01',
              y0=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")].post_covid_error.min(),
              y1=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")].post_covid_error.min(),
              line=dict(color='green', width=2, dash="dash"))
fig.add_annotation(x='2020-01-20',
                  y=energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")].post_covid_error.min(),
                  text=list(energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")].anno_text)[0],
                  showarrow=False,
                  font_size=14,
                  font_family='Arial',
                  font_color=list(energy_data[(energy_data.state_name=='USA')&(energy_data.sector=="RES")].anno_color)[0] 
                  )

updatemenu = []
buttons = []
states = ['USA']
states.extend(energy_data[~(energy_data.state_name =='USA')].state_name.unique())

for state in states:
    buttons.append(dict(method='update',
                        label=state,
                        visible=True,
                        args=[#update to trace
                            {'y':[np.round(energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")&(energy_data.year >= 2019)].pct_error_20,2)],
                               'x':[energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")&(energy_data.year >= 2019)].date],
                               'type':'scatter'},
                            #update to layout
                            {'title':'% Prediction error in residential consumption for '+state,
                             'annotations':[{'x':'2020-01-20',
                                             'y':energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")].post_covid_error.min(),
                                             'text':list(energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")].anno_text)[0],
                                             'showarrow':False,'font_size':14,'font_family':'Arial',
                                             'font_color':list(energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")].anno_color)[0] }],
                            'shapes':[{'type':'line',
                                       'x0':'2019-01-01','x1':'2020-02-15',
                                       'y0':energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")].pre_covid_error.min(),
                                       'y1':energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")].pre_covid_error.min(),
                                       'line':dict(color='red', width=2, dash="dash")},
                                     {'type':'line',
                                       'x0':'2020-02-15','x1':'2020-10-01',
                                       'y0':energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")].post_covid_error.min(),
                                       'y1':energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")].post_covid_error.min(),
                                       'line':dict(color='green', width=2, dash="dash")},
                                     {'type':'line',
                                       'x0':'2020-02-15','x1':'2020-02-15',
                                       'y0':energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")&(energy_data.year >= 2019)].pct_error_20.min()*1.2,
                                       'y1':energy_data[(energy_data.state_name==state)&(energy_data.sector=="RES")&(energy_data.year >= 2019)].pct_error_20.max()*1.2,
                                       'line':dict(color='#f76d23', width=3, dash="dot")}]},
                            [0]]))

    # some adjustments to the updatemenus
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

# add dropdown menus to the figure
fig.update_layout(showlegend=False, updatemenus=updatemenu)
fig.show()

py.plot(fig, filename = 'res-energy-states', auto_open=True)

### Geo plot

In [ ]:
def mean_errors(df, col='pct_error_20'):
    pre_covid = df[(df.date<'2020-03-01')&(df.date >= '2019-01-01')][col].mean()
    post_covid = df[df.date>='2020-03-01'][col].mean()
    diff = post_covid - pre_covid
    return diff
#     return {'pre_covid':pre_covid, 'post_covid':post_covid}

def get_pre_covid_errors(df, col='pct_error_20'):
    pre_covid = df[(df.date<'2020-03-01')&(df.date >= '2019-01-01')][col].mean()
    return pre_covid

def get_post_covid_errors(df, col='pct_error_20'):
    post_covid = df[df.date>='2020-03-01'][col].mean()
    return post_covid

In [ ]:
mean_change = pd.DataFrame(energy_data.groupby(['state','sector']).apply(mean_errors).reset_index().rename(columns={0:'change'}))

In [ ]:
mean_res_change = mean_change[mean_change.sector=="RES"]

In [ ]:
mean_res_change.head()

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'USA':'USA'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
mean_res_change['state_name'] = [abbrev_us_state[s] for s in mean_res_change['state']]

In [ ]:
mean_res_change.head()

In [ ]:
mean_res_change['text'] = [s + ': ' + str(np.round(c,2)) + '%' for s,c in zip(mean_res_change['state_name'],mean_res_change['change'])]

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=mean_res_change['state'], # Spatial coordinates
    z = mean_res_change['change'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'viridis',
    text=mean_res_change['text'],
    colorbar_title = "% change",
    marker_line_color='white', # line markers between states
    hoverinfo = "text"
))

fig.update_layout(
    title_text = 'Post-COVID change in residential energy consumption',
    geo_scope='usa', # limite map scope to USA
    geo= dict(projection=go.layout.geo.Projection(type = 'albers usa'))
)

fig.show()

py.plot(fig, filename = 'res-energy-geo', auto_open=True)